In [1]:
import pandas as pd

In [2]:
# Load the dataset, choose the correct sheet, skip the rows of metadata
file_path = 'unwto-tourism-industries-data.xlsx'
df = pd.read_excel(file_path, sheet_name='Tourism Industries', skiprows=5)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1789 entries, 0 to 1788
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   C.                         1562 non-null   float64
 1   S.                         1562 non-null   float64
 2   C. & S.                    1562 non-null   object 
 3   Basic data and indicators  226 non-null    object 
 4   Unnamed: 4                 224 non-null    object 
 5   Unnamed: 5                 1339 non-null   object 
 6   Unnamed: 6                 0 non-null      float64
 7   Unnamed: 7                 0 non-null      float64
 8   Units                      1339 non-null   object 
 9   Notes                      223 non-null    object 
 10  1995                       1339 non-null   object 
 11  1996                       1339 non-null   object 
 12  1997                       1339 non-null   object 
 13  1998                       1339 non-null   objec

In [3]:
df.head(10)

,C.,S.,C. & S.,Basic data and indicators,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Units,Notes,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 38
0,4.0,0.00,4-0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Accommodation for visitors in hotels and simil...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,4.13,4-4.13,NaN,NaN,Number of establishments,NaN,NaN,Units,NaN,...,..,..,..,..,..,..,..,..,..,NaN
3,4.0,4.14,4-4.14,NaN,NaN,Number of rooms,NaN,NaN,Units,NaN,...,..,..,..,..,..,..,..,..,..,NaN
4,4.0,4.15,4-4.15,NaN,NaN,Number of bed-places,NaN,NaN,Units,NaN,...,..,..,..,..,..,..,..,..,..,NaN
5,4.0,4.16,4-4.16,NaN,NaN,Occupancy rate / rooms,NaN,NaN,Percent,NaN,...,..,..,..,..,..,..,..,..,..,NaN
6,4.0,4.17,4-4.17,NaN,NaN,Occupancy rate / bed-places,NaN,NaN,Percent,NaN,...,..,..,..,..,..,..,..,..,..,NaN
7,4.0,4.18,4-4.18,NaN,NaN,Average length of stay,NaN,NaN,Nights,NaN,...,..,..,..,..,..,..,..,..,..,NaN
8,8.0,0.00,8-0,ALBANIA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,Accommodation for visitors in hotels and simil...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/ Source: Business Register.\n2/ Until 2016: ...


In [4]:
# Rename columns using the actual header names
df.rename(columns={
    'Basic data and indicators': 'Country',
    'Unnamed: 5' : 'Indicator'
}, inplace=True)

In [5]:
# Fill missing country names by copying downward
df['Country'] = df['Country'].fillna(method='ffill')


/var/folders/n9/v4jfbhx154l3n6tpqmvtm2bc0000gn/T/ipykernel_6260/970847455.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Country'] = df['Country'].fillna(method='ffill')


In [6]:
# Extract year columns: keep only numeric column names like 1995, 1996
year_columns = [col for col in df.columns if isinstance(col, int)]

In [7]:
# Convert from wide to long format (better for analysis and SQL)
df_long = df.melt(
    id_vars=['Country', 'Indicator'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

In [8]:
# Drop rows with empty or placeholder values
df_long = df_long.dropna(subset=['Value'])
df_long = df_long[df_long['Value'] != '..']

In [9]:
# Convert Value column to numeric
df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')

In [10]:
# Final clean up: drop rows with invalid numbers
df_long = df_long.dropna(subset=['Value'])


In [11]:
# Save cleaned file to csv
df_long.to_csv('./unwto_cleaned_long.csv', index=False)
print('Cleaned dataset saved as unwto_cleaned_long.csv')

Cleaned dataset saved as unwto_cleaned_long.csv


In [13]:
df_long['Indicator'].unique()

array(['Number of rooms', 'Number of bed-places',
       'Occupancy rate / bed-places', 'Number of establishments',
       'Average length of stay', 'Occupancy rate / rooms',
       'Available capacity (bed-places per 1000 inhabitans)'],
      dtype=object)

In [14]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20015 entries, 19 to 50087
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    20015 non-null  object 
 1   Indicator  20015 non-null  object 
 2   Year       20015 non-null  object 
 3   Value      20015 non-null  float64
dtypes: float64(1), object(3)
memory usage: 781.8+ KB


In [16]:
len(df.Country.unique())

226